## Псевдолемматизация

Идея решения достаточно простая: возьмем обучающую выборку и составим по ней словарь окончаний, какое окончание переходит в какое при лемматизации. Затем найдем такие же окончания в тестовой выборке и заменим по словарю, учитывая, из какой части речи это окончание.

In [2]:
import numpy as np
import pandas as pd
import os
from StringIO import StringIO

Считываем обучающую и тестовую выборки.

In [3]:
train_data = pd.read_csv(
    StringIO(''.join(l.replace('+', ',') for l in open('task2_lemmas_train'))), 
    error_bad_lines=False)

Skipping line 102: expected 4 fields, saw 6
Skipping line 229: expected 4 fields, saw 6
Skipping line 297: expected 4 fields, saw 6
Skipping line 373: expected 4 fields, saw 6
Skipping line 399: expected 4 fields, saw 6
Skipping line 410: expected 4 fields, saw 6
Skipping line 414: expected 4 fields, saw 6
Skipping line 488: expected 4 fields, saw 6
Skipping line 501: expected 4 fields, saw 6
Skipping line 508: expected 4 fields, saw 6
Skipping line 563: expected 4 fields, saw 6
Skipping line 622: expected 4 fields, saw 6
Skipping line 712: expected 4 fields, saw 6
Skipping line 751: expected 4 fields, saw 6
Skipping line 787: expected 4 fields, saw 6
Skipping line 804: expected 4 fields, saw 6
Skipping line 825: expected 4 fields, saw 6
Skipping line 886: expected 4 fields, saw 6
Skipping line 919: expected 4 fields, saw 6
Skipping line 1043: expected 4 fields, saw 6
Skipping line 1109: expected 4 fields, saw 6
Skipping line 1141: expected 4 fields, saw 6
Skipping line 1190: expected 

In [4]:
len(train_data)

116477

In [5]:
train_data.head(5)

,Id,X,y,1
0,1,vergognerete,vergognare,V
1,2,amnistiavate,amnistiare,V
2,3,menomazione,menomazione,N
3,4,sfaldavamo,sfaldare,V
4,5,sfodererei,sfoderare,V


In [6]:
test_data = pd.read_csv('task2_lemmas_test')

In [7]:
len(test_data)

29661

In [8]:
test_data.head(5)

,Id,X
0,1,gettonan
1,2,incidentali
2,3,involtino
3,4,lievi
4,5,comunistizzasse


In [9]:
train_data_V = train_data.copy()
train_data_V = train_data.ix[train_data['1'] == 'V']
train_data_V.reset_index(inplace='True', drop=True)

In [10]:
train_data_V.head(5)

,Id,X,y,1
0,1,vergognerete,vergognare,V
1,2,amnistiavate,amnistiare,V
2,4,sfaldavamo,sfaldare,V
3,5,sfodererei,sfoderare,V
4,6,ascondesti,ascondere,V


Составляем словарь из окончаний, соответствующих глаголам (V).

In [11]:
dict_V = {}

In [12]:
for i in range(len(train_data_V['Id'])):
    root = os.path.commonprefix([train_data_V['X'][i], train_data_V['y'][i]])
    key = train_data_V['X'][i][len(root):]
    if key not in dict_V:
        dict_V[key] = train_data_V['y'][i][len(root):]

In [13]:
dict_V

{'': 'e',
 'a': 'ere',
 'ai': 'e',
 'amo': 're',
 'an': 'ire',
 'anno': 'e',
 'ano': 'ere',
 'ate': 're',
 'bbe': '',
 'bber': '',
 'bbero': '',
 'bbi': 'scere',
 'ca': 'rre',
 'can': 'rre',
 'cano': 're',
 'ccia': 're',
 'cciamo': 're',
 'ccian': 're',
 'cciano': 're',
 'cciate': 're',
 'ccio': 're',
 'ce': 'rre',
 'cei': 'rre',
 'cemmo': 'rre',
 'cendo': 'rre',
 'cerai': 'rre',
 'ceran': 'rre',
 'ceranno': 'rre',
 'cerebbe': 'rre',
 'cerebber': 'rre',
 'cerebbero': 'rre',
 'cerei': 'rre',
 'ceremmo': 'rre',
 'ceremo': 'rre',
 'cereste': 'rre',
 'ceresti': 'rre',
 'cerete': 'rre',
 'ceron': 'rre',
 'cerono': 'rre',
 'cer\xc3\xa0': 'rre',
 'cer\xc3\xb2': 'rre',
 'cesse': 're',
 'cesser': 'rre',
 'cessero': 'rre',
 'cessi': 'rre',
 'cessimo': 're',
 'ceste': 'rre',
 'cesti': 're',
 'cete': 'rre',
 'ceva': 'rre',
 'cevamo': 're',
 'cevan': 'rre',
 'cevano': 're',
 'cevate': 'rre',
 'cevi': 're',
 'cevo': 're',
 'ci': 'rre',
 'cia': 'ere',
 'ciamo': 're',
 'cian': 'ere',
 'ciano': 'ere',


Проходимся по всем словам из тестовой выборки и заменяем окончания, руководствуясь словарем. Сначала обрабатываем глаголы (V), потому что их больше всего.

In [14]:
test_data["y"] = ""

In [15]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [16]:
%%time

for i in range(len(test_data['Id'])):
    for j in range(len(test_data['X'][i])):
        if test_data['X'][i][j:] in dict_V:
            test_data['y'][i] = rreplace(test_data['X'][i], test_data['X'][i][j:], dict_V[test_data['X'][i][j:]], 1) + "+V"
            break
            
    if i % 500 == 0:
        print(i)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
CPU times: user 16min 46s, sys: 1.44 s, total: 16min 48s
Wall time: 16min 46s


In [17]:
test_data

,Id,X,y
0,1,gettonan,gettonire+V
1,2,incidentali,incidentaere+V
2,3,involtino,involtare+V
3,4,lievi,lirre+V
4,5,comunistizzasse,comunistizzare+V
5,6,vidimerebbe,vidimare+V
6,7,imbrodan,imbrodire+V
7,8,strillar,
8,9,cifrasti,cifrare+V
9,10,compassavano,compassare+V


Аналогично обрабатываем существительные (N) и прилагательные (A).

In [31]:
train_data_N = train_data.copy()
train_data_N = train_data_N.loc[train_data['1'] == 'N']
train_data_N.reset_index(inplace='True', drop=True)

In [32]:
train_data_N.head(5)

,Id,X,y,1
0,3,menomazione,menomazione,N
1,44,cafè,cafè,N
2,57,magnaccia,magnaccia,N
3,64,crac,crac,N
4,66,baionetta,baionetta,N


In [33]:
dict_N = {}

In [34]:
%%time

for i in range(len(train_data_N['Id'])):
    root = os.path.commonprefix([train_data_N['X'][i], train_data_N['y'][i]])
    key = train_data_N['X'][i][len(root):]
    if key not in dict_N:
        dict_N[key] = train_data_N['y'][i][len(root):]

CPU times: user 776 ms, sys: 16 ms, total: 792 ms
Wall time: 751 ms


In [35]:
dict_N

{'': '',
 '-chiave': 'o-chiave',
 'a': 'o',
 'ces': 'x',
 'e': 'a',
 'e-estati': 'a-estate',
 'e-gol': 'a-gol',
 'e-lavoro': 'a-lavoro',
 'e-ordinanze': 'a-ordinanza',
 'e-paga': 'a-paga',
 'e-partito': 'a-partito',
 'edonne': 'adonna',
 'eforti': 'aforte',
 'ei': 'io',
 'en': 'an',
 'esorti': 'asorte',
 'he': 'a',
 'he-dati': 'a-dati',
 'heforti': 'aforte',
 'hi': 'o',
 'i': 'e',
 'i-chiave': 'o-chiave',
 'i-italia': 'a-italia',
 'i-leninismi': 'o-leninismo',
 'i-paese': 'a-paese',
 'i-quadro': 'e-quadro',
 'i-spia': 'e-spia',
 'i-stati': 'o-stato',
 'iclan': 'oclan',
 'ies': 'y',
 'igruppo': 'ogruppo',
 'ilista': 'olista',
 'imorte': 'omorta',
 'ini': 'o',
 'ini-radar': 'o-radar',
 'ipattuglia': 'opattuglia',
 'iviri': 'oviro',
 'nes': '',
 's': '',
 '\xc3\xa8i': 'io'}

In [36]:
%%time

for i in range(len(test_data['Id'])):
    #if test_data['y'][i] == "":
    for j in range(len(test_data['X'][i])):
        if test_data['X'][i][j:] in dict_N:
            test_data['y'][i] = rreplace(test_data['X'][i], test_data['X'][i][j:], dict_N[test_data['X'][i][j:]], 1) + "+N"
            break
            
    if i % 500 == 0:
        print(i)

0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
CPU times: user 8min 48s, sys: 936 ms, total: 8min 49s
Wall time: 8min 47s


In [37]:
test_data

,Id,X,y
0,1,gettonan,gettonire+V
1,2,incidentali,incidentale+N
2,3,involtino,involtare+V
3,4,lievi,lieve+N
4,5,comunistizzasse,comunistizzassa+N
5,6,vidimerebbe,vidimerebba+N
6,7,imbrodan,imbrodire+V
7,8,strillar,
8,9,cifrasti,cifraste+N
9,10,compassavano,compassare+V


In [18]:
train_data_A = train_data.copy()
train_data_A = train_data_A.loc[train_data['1'] == 'A']
train_data_A.reset_index(inplace='True', drop=True)

In [19]:
train_data_A.head(5)

,Id,X,y,1
0,38,balenieri,baleniero,A
1,49,osanti,osare,A
2,50,rialzata,rialzato,A
3,56,rivaleggiante,rivaleggiare,A
4,62,cospiranti,cospirare,A


In [20]:
dict_A = {}

In [21]:
%%time

for i in range(len(train_data_A['Id'])):
    root = os.path.commonprefix([train_data_A['X'][i], train_data_A['y'][i]])
    key = train_data_A['X'][i][len(root):]
    if key not in dict_A:
        dict_A[key] = train_data_A['y'][i][len(root):]

CPU times: user 896 ms, sys: 20 ms, total: 916 ms
Wall time: 907 ms


In [22]:
dict_A

{'': '',
 'a': 'o',
 'cente': 're',
 'centi': 'rre',
 'e': 'o',
 'ente': 'ire',
 'enti': 'ire',
 'he': 'o',
 'hi': 'o',
 'i': 'o',
 'maggiore': 'grande',
 'nente': 'rre',
 'nenti': 'rre',
 'nte': 're',
 'nti': 're',
 'rice': 'ore',
 'rici': 'ore',
 'vente': 're',
 'venti': 're',
 'vo': 're'}

In [23]:
%%time

for i in range(len(test_data['Id'])):
    #if test_data['y'][i] == "":
    for j in range(len(test_data['X'][i])):
        if test_data['X'][i][j:] in dict_A:
            test_data['y'][i] = rreplace(test_data['X'][i], test_data['X'][i][j:], dict_A[test_data['X'][i][j:]], 1) + "+A"
            break
            
    if i % 500 == 0:
        print(i)

0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
CPU times: user 9min 3s, sys: 892 ms, total: 9min 3s
Wall time: 9min 2s


In [30]:
test_data

,Id,X,y
0,1,gettonan,gettonire+V
1,2,incidentali,incidentalo+A
2,3,involtino,involtare+V
3,4,lievi,lievo+A
4,5,comunistizzasse,comunistizzasso+A
5,6,vidimerebbe,vidimerebbo+A
6,7,imbrodan,imbrodire+V
7,8,strillar,
8,9,cifrasti,cifrasto+A
9,10,compassavano,compassare+V


Некоторые строчки остались пустыми, потому что там при лемматизации просто добавлялась буква. Это всегда одна и та же буква "е", поэтому просто добавим ее к пустым строкам.

In [38]:
%%time

for i in range(len(test_data['Id'])):
    if test_data['y'][i] == "":
        test_data['y'][i] = test_data['X'][i] + 'e+V'

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 18.3 s, sys: 48 ms, total: 18.4 s
Wall time: 18.3 s


In [39]:
test_data

,Id,X,y
0,1,gettonan,gettonire+V
1,2,incidentali,incidentale+N
2,3,involtino,involtare+V
3,4,lievi,lieve+N
4,5,comunistizzasse,comunistizzassa+N
5,6,vidimerebbe,vidimerebba+N
6,7,imbrodan,imbrodire+V
7,8,strillar,strillare+V
8,9,cifrasti,cifraste+N
9,10,compassavano,compassare+V


Удаляем ненужный столбец и записываем результат в файл.

In [40]:
output = test_data.copy()
del output['X']
output.columns = ['Id', 'Category']
output.to_csv("linear2_submission3.csv", sep=',', index=False)